In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# export
import pandas as pd
import numpy as np
from datetime import timedelta

In [3]:
target_forecast_start = pd.to_datetime('2015-11-01')
forecast_length_days = 30

In [4]:
from kaggle_1c_predict_future_sales.trivial_predict import read_train, forecast_last_average, forecast_to_submission

sample_all_time = read_train(n_sample_items=131, sample_random_state=42)
sample_all_time.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
655,2013-01-27,0,25,1119,399.0,1.0
1774,2013-03-01,0,25,4039,249.0,1.0


In [5]:
# vs 2935849
len(sample_all_time)

22274

In [6]:
sample_all_time.agg({'date': ['min', 'max']})

,date
min,2013-01-01
max,2015-12-10


In [7]:
sample_no_future = sample_all_time.loc[sample_all_time['date'] < target_forecast_start]

In [8]:
train_all_time = read_train()
train_no_future  = train_all_time.loc[train_all_time['date'] < target_forecast_start]
train_all_time = None

In [9]:
test = pd.read_csv('raw/test.csv')

def single_fold(sample_all_time, forecast_start, forecast_length_days):
    id_cols = ['item_id', 'shop_id']

    forecast_end = forecast_start + timedelta(days = forecast_length_days)
    forecast_target_raw = sample_all_time.query('date >= @forecast_start and date <= @forecast_end')
    forecast_target_mo = (forecast_target_raw
        .groupby(id_cols, as_index=False)
        .agg({'item_cnt_day': 'sum'})
        .rename({'item_cnt_day': 'item_cnt_month'}, axis=1)
    )

    forecast_target_mo = (test
                          .merge(forecast_target_mo, on=id_cols, how='left')
                          .fillna(0)
                         )

    train_start = sample_all_time.query('date < @forecast_start')

    predicted_mo = forecast_last_average(
        train_start,
        forecast_month_length=30,
        train_interval_len=30,
        target_forecast_start=forecast_start
    )

    # TODO: should be part of what's predicted by the func
    predicted_mo['item_cnt_month'] = predicted_mo['item_cnt_month'].clip(0, 20)
    forecast_target_mo['item_cnt_month'] = forecast_target_mo['item_cnt_month'].clip(0, 20)
    from sklearn.metrics import mean_squared_error 
    diff = (forecast_target_mo
        .merge(predicted_mo, on=id_cols, suffixes=('_target', '_predicted'))
    )
    rmse = mean_squared_error(diff['item_cnt_month_target'], diff['item_cnt_month_predicted'])
    return rmse

single_fold(sample_all_time,
           forecast_start=pd.to_datetime('2014-06-01'),
           forecast_length_days=30)

2.212121212121212

In [10]:
fold_rmses = []
fold_forecast_starts = []
n_folds = 10
fold_shift_days = 40
for delta in range(1, 1+n_folds):
    forecast_start = target_forecast_start - timedelta(days=delta*fold_shift_days)
    rmse = single_fold(sample_all_time,
           forecast_start=forecast_start,
           forecast_length_days=30)
    fold_rmses.append(rmse)
    fold_forecast_starts.append(forecast_start)

In [11]:
fold_rmses

[1.7638888888888888,
 2.265734265734266,
 3.0422535211267605,
 3.0636942675159236,
 13.547169811320755,
 4.6644295302013425,
 3.8983050847457625,
 1.943661971830986,
 2.0357142857142856,
 3.471014492753623]

In [12]:
# 33.44 doesn't square well with LB of 8 or like 11
# 7.26 after clipping is mucho closero
# 10 folds x 20 days was closer (8.02) and faster than 20 folds x 10 days (7.97)
# LB also downed to 1.13 as a result of clipping :/
sum(fold_rmses) / len(fold_rmses)

3.9695866119832592

In [13]:
from sklearn.base import RegressorMixin

In [14]:
class ForecastLastAverage(RegressorMixin):
    def fit(self, X, y):
        self.train_interval_len_days = (X['date'].max() - X['date'].min()).days
        forecast = (X
            .groupby(['item_id', 'shop_id'],
                    as_index=False)
            .agg({'item_cnt_day': ['sum', 'count'],
                  'date': ['min', 'max']})
        )
        forecast['sale_days'] = (forecast['date']['max']-forecast['date']['min']).dt.days + 1
        forecast['avg_daily_sale_items'] = forecast['item_cnt_day']['sum'] / self.train_interval_len_days
        self.forecast = forecast[['item_id', 'shop_id', 'avg_daily_sale_items']].droplevel(1, axis=1)
        self.na_forecast = 0
        return self

    def predict(self, X):
        return (X
                .merge(self.forecast, how='left', on=['item_id', 'shop_id'])
                .fillna(0))

    def transform(self, X):
        return self.predict(X)

In [15]:
example = pd.DataFrame(
    columns=['date', 'item_id', 'shop_id', 'item_cnt_day'],
    data=[
        ['2010-01-01', 1, 1, 2],
        ['2010-01-01', 1, 2, 3],
        ['2010-01-01', 2, 1, 1],
        ['2010-01-01', 3, 1, 1],
        ['2010-01-06', 3, 1, 2],
    ]
)

example_test = pd.DataFrame(
    columns=['date', 'item_id', 'shop_id'],
    data=[
        ['2010-02-06', 3, 1],
        ['2010-02-06', 2, 1],
        ['2010-02-06', 1, 3],
        ['2010-02-07', 3, 1],
        ['2010-03-07', 3, 1],
    ]
)

example['date'] = pd.to_datetime(example['date'])
reg = ForecastLastAverage()
reg.fit(example, example['item_cnt_day'])
predicted_daily = reg.predict(example_test)
predicted_daily

,date,item_id,shop_id,avg_daily_sale_items
0,2010-02-06,3,1,0.6
1,2010-02-06,2,1,0.2
2,2010-02-06,1,3,0.0
3,2010-02-07,3,1,0.6
4,2010-03-07,3,1,0.6


In [16]:
# aggs in transformers is a deviation from intended use by virtue of being sample size changing transform
# https://github.com/scikit-learn/scikit-learn/issues/3855
# https://github.com/scikit-learn/scikit-learn/issues/4143
# https://scikit-learn-enhancement-proposals.readthedocs.io/en/latest/slep001/proposal.html#slep001-transformers-that-modify-their-target
# Note to self: native model might be to introduce new aggregated features into the data set rather than sampling
# Also: Pipelines only transform the observed data (X).
from sklearn.base import TransformerMixin, BaseEstimator
class MonthlyAggregation(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        tmp = X.copy()
        tmp['date'] = pd.to_datetime(tmp['date'])
        tmp['year_month'] = tmp['date'].dt.strftime('%Y-%m')
        return (tmp.groupby(
            ['item_id', 'shop_id', 'year_month'], as_index=False)
            .agg({'avg_daily_sale_items': 'sum'})
            .rename({'avg_daily_sale_items': 'item_cnt_month'}, axis=1)
        )
    def predict(self, X):
        return self.transform(X)

In [17]:
monthly_agg = MonthlyAggregation()
monthly_agg.fit_transform(predicted_daily)

,item_id,shop_id,year_month,item_cnt_month
0,1,3,2010-02,0.0
1,2,1,2010-02,0.2
2,3,1,2010-02,1.2
3,3,1,2010-03,0.6


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(
    ForecastLastAverage(),
    MonthlyAggregation()
)
pipe.fit(example, example['item_cnt_day'])

Pipeline(steps=[('forecastlastaverage',
                 <__main__.ForecastLastAverage object at 0x7fb8606f75b0>),
                ('monthlyaggregation', MonthlyAggregation())])

In [19]:
pipe.predict(example_test)

,item_id,shop_id,year_month,item_cnt_month
0,1,3,2010-02,0.0
1,2,1,2010-02,0.2
2,3,1,2010-02,1.2
3,3,1,2010-03,0.6


In [20]:
categorical_cols = ['item_id', 'shop_id']
feature_cols = categorical_cols
target_col = 'item_cnt_day'

In [21]:
target_forecast_start = pd.to_datetime('2015-11-01')

from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(sample_all_time[['item_id', 'shop_id']], sample_all_time['item_cnt_day'])
regressor.predict(test[feature_cols])

array([1.56689527, 1.55659079, 1.5597586 , ..., 1.08638581, 0.94470832,
       1.6248404 ])

In [22]:
november_days = [target_forecast_start + timedelta(days=i) for i in range(0, 30)]

In [23]:
def cross_join(left, right):
    left['_tmp_col'] = 0
    right['_tmp_col'] = 0
    return left.merge(right, on='_tmp_col').drop(['_tmp_col'], axis=1)

test_dates = cross_join(test, pd.DataFrame({'date': november_days}))

predicted_daily = test_dates.copy()
predicted_daily[target_col] = regressor.predict(test_dates[feature_cols])
predicted_daily

,ID,shop_id,item_id,date,item_cnt_day
0,0,5,5037,2015-11-01,1.566895
1,0,5,5037,2015-11-02,1.566895
2,0,5,5037,2015-11-03,1.566895
3,0,5,5037,2015-11-04,1.566895
4,0,5,5037,2015-11-05,1.566895
...,...,...,...,...,...
6425995,214199,45,969,2015-11-26,1.624840
6425996,214199,45,969,2015-11-27,1.624840
6425997,214199,45,969,2015-11-28,1.624840
6425998,214199,45,969,2015-11-29,1.624840


In [24]:
predicted_mo = (predicted_daily.groupby(
        ['item_id', 'shop_id'], as_index=False)
        .agg({target_col: 'sum'})
        .rename({target_col: 'item_cnt_month'}, axis=1)
)

In [25]:
submission = forecast_to_submission(predicted_mo)

In [26]:
# Score: 37.25347
submission.to_csv('submissions/submission_0100_linreg_item_shop.csv', index=False)

In [27]:
submission['item_cnt_month'] = submission['item_cnt_month'].clip(0, 20)

In [28]:
# Score: 19.75361
# item_id and shop_id need to be transformed 🤦‍♀️
submission.to_csv('submissions/submission_0101_linreg_item_shop_clipped.csv', index=False)

In [29]:
import pandas as pd
# Local score: 76.17766593875596
# from sklearn.linear_model import LinearRegression
# SGD is much faster and close 75.59508733523127
from sklearn.linear_model import SGDRegressor

regressor = SGDRegressor()

test = pd.read_csv('raw/test.csv')

# TODO: add item_id back
def extract_features(sample_all_time, extraction_target, categorical_cols=['shop_id']):
    from sklearn.preprocessing import OneHotEncoder
    from scipy import sparse

    onehot_encoder = OneHotEncoder(handle_unknown='ignore')
    onehot_encoder.fit(sample_all_time[categorical_cols])
    categorical_features = onehot_encoder.transform(extraction_target[categorical_cols])
    days_since = (extraction_target['date']-pd.to_datetime('2012-01-01')).dt.days
    days_since = days_since.to_numpy()[:,np.newaxis]
    continuous_features = [days_since]
    # Note to self: np would complain about sparse matrix being onedimensional :/
    transformed_features = sparse.hstack([categorical_features] + continuous_features)
    
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler(with_mean=False)

    # Otherwise it's in COO and doesn't support indexing :/
    # Note to self: adding a new feature hurt SGD convergence which I initially misdiagnosed for sparse matrix mgmt issues
    transformed_features = transformed_features.tocsr()
    transformed_features = scaler.fit_transform(transformed_features)
    
    return transformed_features

def single_fold(regressor,
                transformed_features,
                dates_frame,
                ids_frame,
                target_frame,
                forecast_start,
                forecast_length_days):
    # split
    forecast_end_date = forecast_start + timedelta(days = forecast_length_days)
    train_indexes = dates_frame['date'] < forecast_start
    test_indexes = (dates_frame['date'] >= forecast_start) & (dates_frame['date'] <= forecast_end_date)

    grouper = ids_frame['item_id']*100 + ids_frame['shop_id']
    forecast_target_mo = (target_frame
        .groupby(grouper, as_index=False)
        .agg({target_col: 'sum'})
        .rename({target_col: 'item_cnt_month'}, axis=1)
    )

    train_features = transformed_features[train_indexes]
    test_features = transformed_features[test_indexes]

    regressor.fit(train_features, target_frame[train_indexes].squeeze())
    predicted_daily = (regressor
                       .predict(test_features)
                       .squeeze()
                      )
    predicted_daily = pd.DataFrame({'item_id': ids_frame[test_indexes]['item_id'],
                                    'shop_id': ids_frame[test_indexes]['shop_id'],
                                    target_col: predicted_daily})
    predicted_mo = (predicted_daily.groupby(
            ['item_id', 'shop_id'], as_index=False)
            .agg({target_col: 'sum'})
            .rename({target_col: 'item_cnt_month'}, axis=1)
    )

    # TODO: should be part of what's predicted by the func? (so e.g. test targets are transformed)
    predicted_mo['item_cnt_month'] = predicted_mo['item_cnt_month'].clip(0, 20)
    forecast_target_mo['item_cnt_month'] = forecast_target_mo['item_cnt_month'].clip(0, 20)
    from sklearn.metrics import mean_squared_error 
    predicted_mo['item_cnt_month_target'] = forecast_target_mo['item_cnt_month']
    rmse = mean_squared_error(predicted_mo['item_cnt_month_target'], predicted_mo['item_cnt_month'])

    return rmse, predicted_mo

In [30]:
id_cols = ['item_id', 'shop_id']
dates_frame = sample_all_time[['date']]
target_frame = sample_all_time[['item_cnt_day']]
ids_frame = sample_all_time[id_cols]
transformed_features = extract_features(sample_all_time, sample_all_time)

rmse, predicted_mo = single_fold(regressor, transformed_features, dates_frame, ids_frame, target_frame,
           forecast_start=pd.to_datetime('2014-06-01'),
           forecast_length_days=30)

In [31]:
predicted_mo['diff'] = (predicted_mo['item_cnt_month_target']-predicted_mo['item_cnt_month']).abs()
predicted_mo['diff'].sort_values(ascending=False)

129    20.000000
99     20.000000
101    20.000000
102    20.000000
103    20.000000
         ...    
216     1.000000
215     1.000000
155     1.000000
232     0.998278
246     0.349899
Name: diff, Length: 312, dtype: float64

In [32]:
from sklearn.base import clone
def ts_cv(
    orig_regressor,
    train_all_time,
    transformed_features,
    n_folds,
    forecast_length_days=30,
    target_forecast_start = pd.to_datetime('2015-11-01')
):
    id_cols = ['item_id', 'shop_id']
    dates_frame = train_all_time[['date']]
    target_frame = train_all_time[['item_cnt_day']]
    ids_frame = train_all_time[id_cols]

    target_forecast_start = pd.to_datetime('2015-11-01')
    forecast_length_days = 30

    fold_rmses = []
    fold_forecast_starts = []
    n_folds = 3
    fold_shift_days = 40
    for delta in range(1, 1+n_folds):
        forecast_start = target_forecast_start - timedelta(days=delta*fold_shift_days)
        rmse, predicted_mo = single_fold(clone(orig_regressor), transformed_features, dates_frame, ids_frame, target_frame,
               forecast_start=forecast_start,
               forecast_length_days=30)
        fold_rmses.append(rmse)
        fold_forecast_starts.append(forecast_start)
        
    return fold_rmses, fold_forecast_starts

In [33]:
fold_rmses, fold_forecast_starts = ts_cv(regressor, sample_all_time, transformed_features, 3)

In [34]:
sum(fold_rmses) / len(fold_rmses)

157.35189184200814

In [35]:
"""
### just one_hot(item_id), no scaling
-- Epoch 7
Norm: 6.56, NNZs: 60, Bias: 0.586401, T: 20277474, Avg. loss: 3.438490
Total training time: 7.31 seconds.
Convergence after 7 epochs took 7.31 seconds

### without scaling
-- Epoch 1
Norm: 13365889676.59, NNZs: 61, Bias: -832110.113639, T: 2896782, Avg. loss: 146950322552603541504.000000
Total training time: 1.17 seconds.
...

Note to self: overall, tinkering with LR and such would be a bit more productive in e.g. pytorch
"""
#regressor = SGDRegressor(max_iter=10, verbose=15, learning_rate='adaptive')#, penalty='elasticnet')


'\n### just one_hot(item_id), no scaling\n-- Epoch 7\nNorm: 6.56, NNZs: 60, Bias: 0.586401, T: 20277474, Avg. loss: 3.438490\nTotal training time: 7.31 seconds.\nConvergence after 7 epochs took 7.31 seconds\n\n### without scaling\n-- Epoch 1\nNorm: 13365889676.59, NNZs: 61, Bias: -832110.113639, T: 2896782, Avg. loss: 146950322552603541504.000000\nTotal training time: 1.17 seconds.\n...\n\nNote to self: overall, tinkering with LR and such would be a bit more productive in e.g. pytorch\n'

In [36]:
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)

class FeatureExtractor():
    def __init__(self, categorical_cols=['shop_id']):
        self.onehot_encoder = OneHotEncoder(handle_unknown='ignore')
        self.categorical_cols = categorical_cols
        
    def fit(self, sample_all_time):
        derived = self._ensure_derived(sample_all_time)
        self.onehot_encoder.fit(derived[self.categorical_cols])

    def _ensure_derived(self, data):
        for categorical_name in self.categorical_cols:
            if ':' in categorical_name:
                split_char_idx = categorical_name.index(':')
                raw_col_name = categorical_name[:split_char_idx]
                derived_col_name = categorical_name[split_char_idx+1:]
                data[categorical_name] = getattr(data[raw_col_name].dt, derived_col_name)
                        
        return data
        
    def transform(self, extraction_target):                
        derived = self._ensure_derived(extraction_target)
        categorical_features = self.onehot_encoder.transform(derived[self.categorical_cols])
        days_since = (extraction_target['date']-pd.to_datetime('2012-01-01')).dt.days
        days_since = days_since.to_numpy()[:,np.newaxis]
        continuous_features = [days_since]
        # Note to self: np would complain about sparse matrix being onedimensional :/
        transformed_features = sparse.hstack([categorical_features] + continuous_features)
        # Otherwise it's in COO and doesn't support indexing :/
        # Note to self: adding a new feature hurt SGD convergence which I initially misdiagnosed for sparse matrix mgmt issues
        transformed_features = transformed_features.tocsr()
        transformed_features = scaler.fit_transform(transformed_features)
        return transformed_features

In [37]:
class Pipeline():
    def __init__(self, regressor, feature_extractor,
                 daily_train,
                 daily_test,
                 target_forecast_start=pd.to_datetime('2015-11-01')):
        self.regressor = regressor
        self.feature_extractor = feature_extractor
        count_after = len(daily_train.query('date >= @target_forecast_start'))
        if count_after > 0:
            raise ValueError(f"Expected no entries after target forecast start {target_forecast_start}, got {count_after}")
        self.train_no_future = daily_train
        self.test_dates = daily_test
        
    def ensure_train_features(self):
        if not hasattr(self, 'transformed_features'):
            self.feature_extractor.fit(self.train_no_future)
            self.transformed_features = self.feature_extractor.transform(self.train_no_future) 

    def cv(self, n_folds=3):
        self.ensure_train_features()
        self.cv_fold_rmses, self.cv_fold_forecast_starts = ts_cv(
            self.regressor, self.train_no_future, self.transformed_features, n_folds)
        return sum(self.cv_fold_rmses) / len(self.cv_fold_rmses)
        
    def fit(self):
        self.ensure_train_features()
        self.test_features = self.feature_extractor.transform(self.test_dates)
        self.regressor.fit(self.transformed_features, self.train_no_future['item_cnt_day'])

    def predict(self):
        predicted_daily = self.regressor.predict(self.test_features)
        self.predicted_daily = pd.DataFrame({'item_id': self.test_dates['item_id'],
                                             'shop_id': self.test_dates['shop_id'],
                                              target_col: predicted_daily})
        self.predicted_mo = (self.predicted_daily.groupby(
                ['item_id', 'shop_id'], as_index=False)
                .agg({target_col: 'sum'})
                .rename({target_col: 'item_cnt_month'}, axis=1)
        )        
        
    def prepare_submit(self, name, force=False):
        predicted_mo = self.predicted_mo.copy()
        predicted_mo['item_cnt_month'] = predicted_mo['item_cnt_month'].clip(0, 20)
        self.submission = forecast_to_submission(predicted_mo)
        path = f'submissions/submission_{name}.csv'
        import os;
        if os.path.isfile(path) and not force:
            raise ValueError(path + ' already exists.')
        self.submission.to_csv(path, index=False)

In [38]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

In [39]:
pipeline = Pipeline(
    LinearRegression(),
    FeatureExtractor(categorical_cols=['shop_id', 'item_id']),
    sample_no_future, test_dates)
pipeline.cv()

108.90146874585946

In [40]:
pipeline.fit()
pipeline.predict()
pipeline.prepare_submit('0104_linreg_shop_id_days_since', force=True)

In [41]:
"""
Pipeline(
    LinearRegression(),
    FeatureExtractor(categorical_cols=['shop_id']),
    sample_all_time, test_dates)
"""
# TODO: test expected to fail

"\nPipeline(\n    LinearRegression(),\n    FeatureExtractor(categorical_cols=['shop_id']),\n    sample_all_time, test_dates)\n"

In [42]:
pipeline = Pipeline(
    LinearRegression(),
    FeatureExtractor(categorical_cols=['shop_id', 'item_id']),
    train_no_future, test_dates)
pipeline.cv(3)

72.01573293652204

In [43]:
pipeline = Pipeline(
    LinearRegression(),
    FeatureExtractor(categorical_cols=['shop_id']),
    train_no_future, test_dates)
pipeline.cv(3)

68.57399120343979

In [44]:
pipeline = Pipeline(
    LinearRegression(),
    FeatureExtractor(categorical_cols=['shop_id', 'date:dayofweek']),
    train_no_future, test_dates)
pipeline.cv(3)

68.5705214688219

In [45]:
pipeline.fit()
pipeline.predict()
# Score: 
#pipeline.prepare_submit('0105_linreg_shop_id_days_since_dayof_week')

In [46]:
pipeline.feature_extractor.categorical_cols

['shop_id', 'date:dayofweek']

In [50]:
pipeline.predicted_mo

,item_id,shop_id,item_cnt_month
0,30,2,20.0
1,30,3,20.0
2,30,4,20.0
3,30,5,20.0
4,30,6,20.0
...,...,...,...
214195,22167,55,20.0
214196,22167,56,20.0
214197,22167,57,20.0
214198,22167,58,20.0


In [47]:
# Score: 2.70899
#submission['item_cnt_month'] =submission['item_cnt_month'].clip(0, 20)
#submission.to_csv('submissions/submission_0102_linreg_item_shop_clipped.csv', index=False)

In [48]:
# Score: 1.42040
#submission['item_cnt_month'] =submission['item_cnt_month'].clip(0, 20)
#submission.to_csv('submissions/submission_0103_sgd_linreg_item_shop_clipped.csv', index=False)